In [2]:
pip install psycopg2-binary

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np

import re

from sqlalchemy import create_engine
import psycopg2


In [4]:
# create a function that takes in two datasets. function called etl(extract/transform/load).
def extract_transform_load():
    #read in framingham CSV and extention excel files
    framingham_df = pd.read_csv('framingham.csv')
    extention_df = pd.read_excel('extention of Z-Alizadeh sani dataset.xlsx')
    return framingham_df, extention_df

#create the path to files
file_dir = '/Users/marthamejia/Desktop/coding class/Project CAD'
# framingham data
framingham_file = f'{file_dir}/framingham.csv'
# extention data
extention_file = f'{file_dir}/extention of Z-Alizadeh sani dataset.xlsx'

#set the variables to the ETL function
framingham_file, extention_file = extract_transform_load()

In [5]:
#set dataframe equal to file names
framingham_df = framingham_file
extention_df = extention_file

# Cleaning up Z-Alizadeh data

In [6]:
extention_df.head()

,Age,Weight,Length,Sex,BMI,DM,HTN,Current Smoker,EX-Smoker,FH,...,Lymph,Neut,PLT,EF-TTE,Region RWMA,VHD,LAD,LCX,RCA,Cath
0,53,90,175,Male,29.387755,0,1,1,0,0,...,39,52,261,50,0,N,Stenotic,Normal,Stenotic,CAD
1,67,70,157,Fmale,28.398718,0,1,0,0,0,...,38,55,165,40,4,N,Stenotic,Stenotic,Normal,CAD
2,54,54,164,Male,20.077335,0,0,1,0,0,...,38,60,230,40,2,mild,Stenotic,Normal,Normal,CAD
3,66,67,158,Fmale,26.838648,0,1,0,0,0,...,18,72,742,55,0,Severe,Normal,Normal,Normal,Normal
4,50,87,153,Fmale,37.165193,0,1,0,0,0,...,55,39,274,50,0,Severe,Normal,Normal,Normal,Normal


In [7]:
# create totChol column 

extention_df['totChol']= extention_df.LDL + extention_df.HDL + (0.2*extention_df.TG)

# Add location lat= 35.941286466480605, long= 51.543829732533034
extention_df['lat']= '35.941286466480605' 
extention_df['long']= '51.543829732533034' 

In [8]:
extention_df.head()


,Age,Weight,Length,Sex,BMI,DM,HTN,Current Smoker,EX-Smoker,FH,...,EF-TTE,Region RWMA,VHD,LAD,LCX,RCA,Cath,totChol,lat,long
0,53,90,175,Male,29.387755,0,1,1,0,0,...,50,0,N,Stenotic,Normal,Stenotic,CAD,235.0,35.941286466480605,51.543829732533034
1,67,70,157,Fmale,28.398718,0,1,0,0,0,...,40,4,N,Stenotic,Stenotic,Normal,CAD,218.8,35.941286466480605,51.543829732533034
2,54,54,164,Male,20.077335,0,0,1,0,0,...,40,2,mild,Stenotic,Normal,Normal,CAD,135.6,35.941286466480605,51.543829732533034
3,66,67,158,Fmale,26.838648,0,1,0,0,0,...,55,0,Severe,Normal,Normal,Normal,Normal,94.6,35.941286466480605,51.543829732533034
4,50,87,153,Fmale,37.165193,0,1,0,0,0,...,50,0,Severe,Normal,Normal,Normal,Normal,194.0,35.941286466480605,51.543829732533034


In [9]:
extention_df.drop(columns= ['Weight','Length','EX-Smoker','FH','Obesity','CRF','CVA','Airway disease','Thyroid Disease',
                            'CHF','DLP','Edema','Weak Peripheral Pulse', 'Lung rales', 'Systolic Murmur', 
                            'Diastolic Murmur','Typical Chest Pain','Dyspnea','Function Class','Atypical',
                            'Nonanginal','Exertional CP','LowTH Ang', 'Q Wave','St Elevation','St Depression',
                            'Tinversion','LVH','Poor R Progression','BBB','CR','TG','LDL','HDL','BUN','ESR','HB','K',
                            'Na','WBC','Lymph','Neut','PLT','EF-TTE','Region RWMA','VHD','LAD','LCX','RCA','Cath'], axis = 1, inplace = True)
extention_df.head()

,Age,Sex,BMI,DM,HTN,Current Smoker,BP,PR,FBS,totChol,lat,long
0,53,Male,29.387755,0,1,1,110,80,90,235.0,35.941286466480605,51.543829732533034
1,67,Fmale,28.398718,0,1,0,140,80,80,218.8,35.941286466480605,51.543829732533034
2,54,Male,20.077335,0,0,1,100,100,85,135.6,35.941286466480605,51.543829732533034
3,66,Fmale,26.838648,0,1,0,100,80,78,94.6,35.941286466480605,51.543829732533034
4,50,Fmale,37.165193,0,1,0,110,80,104,194.0,35.941286466480605,51.543829732533034


In [10]:
#Make new column converting 'Sex' to yes=1 and no=0 and naming column 'male' 
extention_df['male']=extention_df['Sex'].apply(lambda x: 1 if x== 'Male' else 0)
extention_df.head()

,Age,Sex,BMI,DM,HTN,Current Smoker,BP,PR,FBS,totChol,lat,long,male
0,53,Male,29.387755,0,1,1,110,80,90,235.0,35.941286466480605,51.543829732533034,1
1,67,Fmale,28.398718,0,1,0,140,80,80,218.8,35.941286466480605,51.543829732533034,0
2,54,Male,20.077335,0,0,1,100,100,85,135.6,35.941286466480605,51.543829732533034,1
3,66,Fmale,26.838648,0,1,0,100,80,78,94.6,35.941286466480605,51.543829732533034,0
4,50,Fmale,37.165193,0,1,0,110,80,104,194.0,35.941286466480605,51.543829732533034,0


In [11]:
extention_df.drop('Sex', axis= 1, inplace= True)
extention_df.head()

,Age,BMI,DM,HTN,Current Smoker,BP,PR,FBS,totChol,lat,long,male
0,53,29.387755,0,1,1,110,80,90,235.0,35.941286466480605,51.543829732533034,1
1,67,28.398718,0,1,0,140,80,80,218.8,35.941286466480605,51.543829732533034,0
2,54,20.077335,0,0,1,100,100,85,135.6,35.941286466480605,51.543829732533034,1
3,66,26.838648,0,1,0,100,80,78,94.6,35.941286466480605,51.543829732533034,0
4,50,37.165193,0,1,0,110,80,104,194.0,35.941286466480605,51.543829732533034,0


In [12]:
cleaned_extention_df = extention_df.rename(columns={'Age': 'age', 'DM': 'diabetes','BP': 'sysBP', 'Current Smoker': 'currentSmoker',
                                                    'FBS': 'glucose', 'PR': 'heartRate', 'HTN': 'prevalentHyp'})
cleaned_extention_df.head()

,age,BMI,diabetes,prevalentHyp,currentSmoker,sysBP,heartRate,glucose,totChol,lat,long,male
0,53,29.387755,0,1,1,110,80,90,235.0,35.941286466480605,51.543829732533034,1
1,67,28.398718,0,1,0,140,80,80,218.8,35.941286466480605,51.543829732533034,0
2,54,20.077335,0,0,1,100,100,85,135.6,35.941286466480605,51.543829732533034,1
3,66,26.838648,0,1,0,100,80,78,94.6,35.941286466480605,51.543829732533034,0
4,50,37.165193,0,1,0,110,80,104,194.0,35.941286466480605,51.543829732533034,0


In [13]:
# Saving cleaned data to use for ml
cleaned_extention_df.to_csv('cleaned_extention.csv')

# Cleaning up Framingham data

In [14]:
framingham_df.head()

,ID,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,2,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,3,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,4,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,5,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [15]:
# Add location lat= 42.27862022222429, long= -71.41820304365233
framingham_df['lat']= '42.27862022222429' 
framingham_df['long']= '-71.41820304365233' 

In [16]:
framingham_df.head()

,ID,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD,lat,long
0,1,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0,42.27862022222429,-71.41820304365233
1,2,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0,42.27862022222429,-71.41820304365233
2,3,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0,42.27862022222429,-71.41820304365233
3,4,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1,42.27862022222429,-71.41820304365233
4,5,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0,42.27862022222429,-71.41820304365233


In [17]:
#drop columns not needed
framingham_df.drop(columns=['ID','education', 'cigsPerDay', 'BPMeds', 'prevalentStroke'], axis= 1, inplace= True)
framingham_df.head()

,male,age,currentSmoker,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD,lat,long
0,1,39,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0,42.27862022222429,-71.41820304365233
1,0,46,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0,42.27862022222429,-71.41820304365233
2,1,48,1,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0,42.27862022222429,-71.41820304365233
3,0,61,1,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1,42.27862022222429,-71.41820304365233
4,0,46,1,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0,42.27862022222429,-71.41820304365233


In [18]:
# Saving cleaned data to use for ml
framingham_df.to_csv('cleaned_framingham.csv')

#  Creating database with both datasets 

In [20]:
ml_df= framingham_df.append(cleaned_extention_df)
ml_df

,male,age,currentSmoker,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD,lat,long
0,1,39,0,0,0,195.0,106.0,70.0,26.970000,80.0,77.0,0.0,42.27862022222429,-71.41820304365233
1,0,46,0,0,0,250.0,121.0,81.0,28.730000,95.0,76.0,0.0,42.27862022222429,-71.41820304365233
2,1,48,1,0,0,245.0,127.5,80.0,25.340000,75.0,70.0,0.0,42.27862022222429,-71.41820304365233
3,0,61,1,1,0,225.0,150.0,95.0,28.580000,65.0,103.0,1.0,42.27862022222429,-71.41820304365233
4,0,46,1,0,0,285.0,130.0,84.0,23.100000,85.0,85.0,0.0,42.27862022222429,-71.41820304365233
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,1,58,0,0,0,181.4,100.0,NaN,29.761905,76.0,92.0,NaN,35.941286466480605,51.543829732533034
299,0,55,0,0,0,85.2,100.0,NaN,27.700831,60.0,86.0,NaN,35.941286466480605,51.543829732533034
300,0,48,0,1,0,172.6,130.0,NaN,30.078125,70.0,83.0,NaN,35.941286466480605,51.543829732533034
301,0,57,0,0,1,202.2,100.0,NaN,35.599858,60.0,96.0,NaN,35.941286466480605,51.543829732533034


In [21]:
# Saving combined data to use for ml
ml_df.to_csv('ml.csv')